# Day 2 - Classifying embeddings with Keras

このノートブックでは、Gemini APIによって生成された埋め込みを使用して、ニュースグループの投稿を投稿内容からカテゴリ（ニュースグループ自体）に分類できるモデルをトレーニングする方法を学びます。

この手法は、Gemini APIの埋め込みを入力として使用し、テキスト入力を直接トレーニングする必要がなくなり、その結果、テキストモデルをゼロからトレーニングするよりも、比較的少ない例を使用してかなりうまく実行できます。

* テキストからメールアドレスを削除できるようになる
* genaiを使って、分類タスク用の埋め込みを作成できるようになる
* kerasモデルを構築できるようになる


# Set up  the SDK

In [1]:
!pip install -qU google-genai

from google import genai
from google.genai import types

genai.__version__

'1.15.0'

In [2]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]

client = genai.Client(vertexai = True, location = "us-central1")

# Dataset

[20のニュースグループテキストデータセット](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)には、トレーニングセットとテストセットに分けられた20のトピックに関する18,000のニュースグループ投稿が含まれています。トレーニングデータセットとテストデータセットの分割は、特定の日付の前後に投稿されたメッセージに基づいています。このチュートリアルでは、トレーニングセットとテストセットのサンプルサブセットを使用し、Pandasを使用していくつかの処理を実行します。

In [3]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset = "train")
newsgroups_test = fetch_20newsgroups(subset="test")

# View list of class names for dataset
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

---
以下は、トレーニングセットのレコードがどのように見えるかの例です。

In [4]:
print(newsgroups_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







Pandasデータフレームでこのチュートリアルのデータを前処理することから始めます。名前やメールアドレスなどの機密情報を削除するには、各メッセージの件名と本文のみを削除します。これはオプションのステップで、入力データを電子メールの投稿ではなく、より一般的なテキストに変換して、他のコンテキストでも機能します。

In [5]:
import email
import re
import pandas as pd

def preprocess_newsgroup_row(data):
    # Extract only the subject and body
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # Strip any remaining email addresses
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # Truncate each entry to 5,000 characters
    return text

def preprocess_newsgroup_data(newsgroup_dataset):
    # Put data points into dataframe
    df = pd.DataFrame(
        {"Text" : newsgroup_dataset.data, "Label" : newsgroup_dataset.target}
    )
    # Clean up the text
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # Match label to target name index
    df["Class Name"] = df["Label"].map(lambda l : newsgroup_dataset.target_names[l])
    return df

In [6]:
# Apply preprocessing function to training and test datasets
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)

df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


次に、トレーニングデータセットで100のデータポイントを取り、いくつかのカテゴリをドロップしてこのチュートリアルを実行して、データの一部をサンプリングします。比較する科学のカテゴリーを選択してください。

In [8]:
def sample_data(df, num_samples, classes_to_keep):
    # Sample rows, selecting num_samples of each Label.
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )
    
    df = df[df["Class Name"].str.contains(classes_to_keep)]
    
    # We have fewer categories now, so re-calibrate the label encoding.
    df["Class Name"] = df["Class Name"].astype("category")
    df["Encoded Label"] = df["Class Name"].cat.codes

    return df

In [9]:
TRAIN_NUM_SAMPLES = 100
TEST_NUM_SAMPLES = 25
# Class name should contain 'sci' to keep science categories.
# Try different labels from the data - see newsgroups_train.target_names
CLASSES_TO_KEEP = "sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

In [10]:
df_train.value_counts("Class Name")

Class Name
sci.crypt          100
sci.electronics    100
sci.med            100
sci.space          100
Name: count, dtype: int64

In [11]:
df_test.value_counts("Class Name")

Class Name
sci.crypt          25
sci.electronics    25
sci.med            25
sci.space          25
Name: count, dtype: int64

# 埋め込みEmbeddingの作成
このセクションでは、Gemini API埋め込みエンドポイントを使用して、各テキストの埋め込みを生成します。埋め込みの詳細については、[埋め込みガイド](https://ai.google.dev/docs/embeddings_guide)を参照してください。

注：埋め込みは一度に1つずつ計算されるため、サンプルサイズが大きいと時間がかかる可能性があります。

## Task type
Text-embedding-004モデルは、特定のタスクに合わせた埋め込みを生成するタスクタイプパラメータをサポートしています。
Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.
FACT_VERIFICATION | Specifies that the given text will be used for fact verification.

今回はCLASSIFICATION

In [12]:
from google.api_core import retry
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# Add tqdm to Pandas...
tqdmr.pandas()

# ...But suppress the experimental warning.
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e : (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

@retry.Retry(predicate = is_retriable, timeout = 300.0)
def embed_fn(text : str) -> list[float]:
    # You will be performing classification, so set task_type accordingly.
    response = client.models.embed_content(
        model = "text-embedding-004",
        contents = text,
        config = types.EmbedContentConfig(
            task_type = "CLASSIFICATION"
        )
    )
    return response.embeddings[0].values

def create_embeddings(df):
    df["Embeddings"] = df["Text"].progress_apply(embed_fn)
    return df

このコードは明確化のために最適化されており、特に高速ではありません。これは、読者が[バッチ](https://ai.google.dev/api/embeddings#method:-models.batchembedcontents)または並列/非同期埋め込み生成を実装するための演習として残されています。このステップを実行するには時間がかかります。

In [13]:
df_train = create_embeddings(df_train)
df_test = create_embeddings(df_test)

Output()

Output()

In [14]:
df_train.head()

,Text,Label,Class Name,Encoded Label,Embeddings
1100,Privacy & Anonymity on the Internet FAQ (2 of ...,11,sci.crypt,0,"[-0.005880952347069979, 0.013432726263999939, ..."
1101,Re: text of White House announcement and Q&As ...,11,sci.crypt,0,"[-0.0086032934486866, 0.03312867134809494, -0...."
1102,"Re: Once tapped, your code is no good any more...",11,sci.crypt,0,"[0.0007966440753079951, 0.02588818222284317, -..."
1103,Clipper considered harmful\n\nIf Clipper comes...,11,sci.crypt,0,"[0.026746241375803947, 0.026608575135469437, -..."
1104,Re: What the clipper nay-sayers sound like to ...,11,sci.crypt,0,"[0.003544179257005453, 0.023313341662287712, -..."


# 分類モデルを構築する
ここでは、生の埋め込みデータを入力として受け入れ、1つの隠しレイヤーと、クラスの確率を指定する出力レイヤーを持つシンプルなモデルを定義します。予測は、テキストが特定のクラスのニュースである確率に対応します。

モデルを実行すると、Kerasはデータポイントのシャッフル、メトリクスの計算、その他のMLボイラープレートなどの詳細を処理します。

In [15]:
!pip install -q keras tensorflow
import keras
from keras import layers

def build_classification_model(input_size : int, num_classes : int) -> keras.Model:
    return keras.Sequential(
        [
            layers.Input([input_size], name = "embedding_inputs"),
            layers.Dense(input_size, activation = "relu", name = "hidden"),
            layers.Dense(num_classes, activation = "softmax", name = "output_probs"),
        ]
    )

2025-05-16 12:26:04.886285: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 12:26:04.894290: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-16 12:26:04.910216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747398364.935429   15781 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747398364.941591   15781 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747398364.958833   15781 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [22]:
# Derive the embedding size from observing the data. The embedding size can also be specified
# with the `output_dimensionality` parameter to `embed_content` if you need to reduce it.

embedding_size = len(df_train["Embeddings"].iloc[0])# 768

classifier = build_classification_model(
    embedding_size, len(df_train["Class Name"].unique())
)

classifier.summary()

classifier.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ["accuracy"]
)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden (Dense)                  │ (None, 768)            │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_probs (Dense)            │ (None, 4)              │         3,076 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 593,668 (2.26 MB)

 Trainable params: 593,668 (2.26 MB)

 Non-trainable params: 0 (0.00 B)

---
# モデルの学習
最後に、モデルを訓練することができます。このコードは、損失値が安定するとトレーニングループを終了するために早期停止を使用するため、実行されるエポックループの数は指定された値と異なる場合があります。

In [23]:
df_train.head()

,Text,Label,Class Name,Encoded Label,Embeddings
1100,Privacy & Anonymity on the Internet FAQ (2 of ...,11,sci.crypt,0,"[-0.005880952347069979, 0.013432726263999939, ..."
1101,Re: text of White House announcement and Q&As ...,11,sci.crypt,0,"[-0.0086032934486866, 0.03312867134809494, -0...."
1102,"Re: Once tapped, your code is no good any more...",11,sci.crypt,0,"[0.0007966440753079951, 0.02588818222284317, -..."
1103,Clipper considered harmful\n\nIf Clipper comes...,11,sci.crypt,0,"[0.026746241375803947, 0.026608575135469437, -..."
1104,Re: What the clipper nay-sayers sound like to ...,11,sci.crypt,0,"[0.003544179257005453, 0.023313341662287712, -..."


In [31]:
import numpy as np

NUM_EPOCHS = 20
BATCH_SIZE = 32

# Split the x and y components of the train and validation subsets.
x_train = np.stack(df_train["Embeddings"])# (400, 768)
y_train = df_train["Encoded Label"]

x_val = np.stack(df_test["Embeddings"])# (100, 768)
y_val = df_test["Encoded Label"]

# Specify that it's OK to stop early if accuracy stabilises.
early_stop = keras.callbacks.EarlyStopping(monitor = "accuracy", patience = 3)

# Train the model for the desired number of epochs.
history = classifier.fit(
    x = x_train,
    y = y_train,
    validation_data = (x_val, y_val),
    callbacks = [early_stop],
    batch_size = BATCH_SIZE,
    epochs = NUM_EPOCHS,
)

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.3956 - loss: 1.3638 - val_accuracy: 0.8000 - val_loss: 1.2354
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8432 - loss: 1.1709 - val_accuracy: 0.8200 - val_loss: 1.0639
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8865 - loss: 0.9824 - val_accuracy: 0.8700 - val_loss: 0.8873
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9337 - loss: 0.7474 - val_accuracy: 0.8400 - val_loss: 0.7235
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9421 - loss: 0.5799 - val_accuracy: 0.9000 - val_loss: 0.5976
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9582 - loss: 0.4540 - val_accuracy: 0.9400 - val_loss: 0.4908
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9808 - loss: 0.3291 - val_accuracy: 0.8900 - val_loss: 0.4521
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9718 - loss: 0.2577 - val_accuracy: 0.9100 - v

# モデルの評価

In [33]:
classifier.evaluate(x = x_val, y = y_val, return_dict = True)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9262 - loss: 0.2794 


{'accuracy': 0.9300000071525574, 'loss': 0.2567942142486572}

モデルトレーニング指標の視覚化方法など、Kerasを使用したトレーニングモデルの詳細については、[組み込みのメソッドを使用したトレーニングと評価]((https://www.tensorflow.org/guide/keras/training_with_built_in_methods))を参照してください。

# モデルの予測

優れた評価指標を備えたトレーニングされたモデルが手に入ったので、新しい手書きデータで予測を試みることができます。提供された例を使用するか、独自のデータを試して、モデルがどのように機能するかを確認してください。

In [42]:
def make_prediction(text : str) -> list[float]:
    """Infer categories from the provided text."""
    #Remember that the model takes embeddings as input, so calculate them first.
    embedded = embed_fn(new_text)
    # And recall that the input must be batched, so here they are wrapped as a
    # list to provide a batch of 1.
    inp = np.array([embedded])
    
    # And un-batched here.
    [result] = classifier.predict(inp)
    return result

In [47]:
# This example avoids any space-specific terminology to see if the model avoids
# biases towards specific jargon.
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""


result = make_prediction(new_text)

for idx, category in enumerate(df_test["Class Name"].cat.categories):
    print(f"{category}: {result[idx] * 100:0.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
sci.crypt: 0.02%
sci.electronics: 0.30%
sci.med: 0.04%
sci.space: 99.64%


Kerasでカスタムモデルをトレーニングする詳細については、[Kerasガイド](https://keras.io/guides/)をご覧ください。